In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import the API key(s)
from config import geoapify_key

In [9]:
#try zips
#Read in CSV containing all US zip codes
zip_df = pd.read_csv('resources/us_zips.csv', delimiter=";")

#Drop columns other than Zip Code and State, then sort by State while dropping the original index and replacing it with Official State Name
states_clean = zip_df.loc[:,['Official USPS State Code','Official State Name']].sort_values('Official State Name').drop_duplicates().reset_index(drop=True)
#Preview dataframe of state abbrev to full name converting dataframe
states_clean.head

,Official USPS State Code,Official State Name
0,AL,Alabama
1,AK,Alaska
2,AS,American Samoa
3,AZ,Arizona
4,AR,Arkansas
5,CA,California
6,CO,Colorado
7,CT,Connecticut
8,DE,Delaware
9,DC,District of Columbia


In [10]:
# #TRY STATE QUERY
# states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
#            'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
#            'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
#            'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
#            'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
# base_placeid_url = 'https://api.geoapify.com/v1/geocode/search'
# params = {
#     'apiKey': geoapify_key,
#     'format': 'json'
# }
# place_ids = []
# for state in states:
#     params['state'] = state
# #    print(base_placeid_url + '?state='+ state + '&format=json'+ '&apiKey=' + geoapify_key)
#     print(requests.get(base_placeid_url,params))
# #     response = requests.get(base_placeid_url,params).json()
# #     print(response)
# #     try:
# #         place_ID = response['results'][0]['place_id']
# #     except:
# #         place_ID = np.nan
# #     place_ids.append(place_ID)
# # print(place_ids)

In [11]:
#  #TRY ZIP QUERY
# #in order to retrieve locations from the places geoapify, need place IDs
# base_placeid_url = 'https://api.geoapify.com/v1/geocode/search'
# params = {
#     'apiKey': geoapify_key,
#     'format': 'json'
#     #will do 'postcode' in a for loop
# }
# place_ids = []
# for index, row in zip_clean.iterrows():
#     zipcode = row['Zip Code']
#     params['postcode'] = zipcode
#     response = requests.get(base_placeid_url,params).json()
#     try:
#         place_ID = response['results'][0]['place_id']
#     except:
#         place_ID = np.nan
#     place_ids.append(place_ID)
# zip_clean['Place ID'] = place_ids
# print(place_ids)

In [12]:
# # STILL ATTEMPTING ZIP
# #Can now grab the zip codes for each state to be able to organize geoapify calls by state
# #i.e. zip_clean.loc['Alabama']
# #Set up the DataFrame to hold info from API calls
# zip_clean['Sports Facilities'] = ''
# zip_clean['Fast Food'] = ''
# zip_clean

In [55]:
#for looping sports
#api_sport_list = ['sport','activity.sport_club','commercial.outdoor_and_sport','commercial.outdoor_and_sport.ski','commercial.outdoor_and_sport.diving','commercial.outdoor_and_sport.bicycle','commercial.outdoor_and_sport.golf','building.sport']

In [59]:
#for inputing at once
api_sport_list = 'sport,activity.sport_club,commercial.outdoor_and_sport,commercial.outdoor_and_sport.ski,commercial.outdoor_and_sport.diving,commercial.outdoor_and_sport.bicycle,commercial.outdoor_and_sport.golf,building.sport'

In [94]:
#rect:lon1,lat1,lon2,lat2
#splitting US into rectangles based on limits of API call geometry
rectangles = []
lat_us_top = 49.484855
lat_us_bottom = 25.148082
lon_us_left = -126.562500
lon_us_right = -66.414413
lat_int = (lat_us_top - lat_us_bottom)/3
lon_int = (lon_us_left - lon_us_right)/5
lon_num_list = np.arange(1,6,1)
lat_num_list = np.arange(1,4,1)
for _ in lon_num_list:
    for _ in lat_num_list:
        lon2 = lon_us_left + lon_int
        lat2 = lat_us_bottom + lat_int
        rect = f'{lon_us_left},{lat_us_bottom},{lon2},{lat2}'
        rectangles.append(rect)
        lat_us_bottom = lat_us_bottom + lat_int
    lat_us_bottom = 19.50139
    lon_us_left = lon_us_left + lon_int
    #second row of rectangles
print(rectangles)

['-126.5625,25.148082,-138.5921174,33.26033966666667', '-126.5625,33.26033966666667,-138.5921174,41.37259733333333', '-126.5625,41.37259733333333,-138.5921174,49.484854999999996', '-138.5921174,19.50139,-150.6217348,27.61364766666667', '-138.5921174,27.61364766666667,-150.6217348,35.72590533333334', '-138.5921174,35.72590533333334,-150.6217348,43.83816300000001', '-150.6217348,19.50139,-162.65135220000002,27.61364766666667', '-150.6217348,27.61364766666667,-162.65135220000002,35.72590533333334', '-150.6217348,35.72590533333334,-162.65135220000002,43.83816300000001', '-162.65135220000002,19.50139,-174.68096960000003,27.61364766666667', '-162.65135220000002,27.61364766666667,-174.68096960000003,35.72590533333334', '-162.65135220000002,35.72590533333334,-174.68096960000003,43.83816300000001', '-174.68096960000003,19.50139,-186.71058700000003,27.61364766666667', '-174.68096960000003,27.61364766666667,-186.71058700000003,35.72590533333334', '-174.68096960000003,35.72590533333334,-186.710587

In [96]:
base_url = 'https://api.geoapify.com/v2/places'
params={
    #'filter':'circle:-98.35,39.50,4506163'
    #'filter':'circle:-98.350231,39.508968,500000',
    'apiKey': geoapify_key,
    'categories': api_sport_list
}
for rectangle in rectangles:
    params['filter'] = 'rect:' + rectangle
    response = requests.get(base_url,params).json()
    states = []
    list_of_features = response['features']
    if response['features'] == []:
        pass
    else:
        for feature in list_of_features:
            try:
                state = feature['properties']['state']
                state_abbr = feature['properties']['state_code']
                gym_count = 1
            except:
                pass
            states.append(state)

In [97]:
print(states)

['Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community', 'Valencian Community']


In [72]:
base_url = 'https://api.geoapify.com/v2/places'
params={
    #'filter':'circle:-98.35,39.50,4506163',
    'filter':'circle:-98.350231,39.508968,500000',
    'apiKey': geoapify_key,
    'categories': api_sport_list
}
response = requests.get(base_url,params).json()
states = []
list_of_features = response['features']
for feature in list_of_features:
    state = feature['properties']['state']
    state_abbr = feature['properties']['state_code']
    gym_count = 1
    states.append(state)
print(states)

['Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas', 'Kansas']


In [58]:
#build base URL for places
#starting with sports
base_url = 'https://api.geoapify.com/v2/places'
params={
    'filter':'circle:-98.35,39.50,4506163',
    'apiKey': geoapify_key
}
print(requests.get(base_url,params).json())
feats = []
for sport in api_sport_list:
    params['categories'] = sport
    response = requests.get(base_url,params).json()
    print(response)
    break
#     list_of_features = response['features']
#     temp = []
#     for feature in list_of_features:
#         state = feature['properties']['state']
#         state_abbr = feature['properties']['state_code']
#         gym_count = 1
#     temp.append(state)
# feats.append(temp)
            

{'statusCode': 400, 'error': 'Bad Request', 'message': '"value" must contain at least one of [type, categories]'}
{'type': 'FeatureCollection', 'features': []}


In [52]:
#build base URL for places
#starting with sports
base_url = 'https://api.geoapify.com/v2/places'
params={
    'filter':'circle:-98.35,39.50,4506163',
    'apiKey': geoapify_key
}
feats = []
for sport in api_sport_list:
    params['categories'] = sport
    response = requests.get(base_url,params).json()
    list_of_features = response['features']
    print(response)
    temp = []
    for feature in list_of_features:
        try:
            state = feature['properties']['state']
            state_abbr = feature['properties']['state_code']
            gym_count = 1
        except:
            state = np.nan
        temp.append(state)
    feats.append(temp)

{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}


In [ ]:
url = 'https://api.geoapify.com/v2/places?filter=

In [29]:
feats

[[], [], [], [], [], [], []]

In [18]:
#build base URL for places
#starting with fast food
base_url = 'https://api.geoapify.com/v2/places'
params={
    'categories':'sport',
    #filter is rough contiguous US
    'filter':'rect:-66.95,24.52,-124.77,49.38',
    'apiKey': geoapify_key
}
response = requests.get(base_url,params).json()
list_of_features = response['features']
for feature in list_of_features:
    state = feature['properties']['state']
    state_abbr = feature['properties']['state_code']
    gym_count = 1
    print(state,state_abbr)

Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
Kansas KS
